## PE spectra

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import numpy as np
import awkward as ak
import os
import pint
import glob
import re
from functools import partial

# I use the SiPM QC Analysis kernel (from .venv/bin)

u = pint.get_application_registry()

from lgdo import lh5
from lgdo.lh5 import LH5Store, LH5Iterator, read, read_as, ls, show
from dspeed.vis.waveform_browser import WaveformBrowser
from legendmeta import LegendMetadata

from latools.utils import get_key_for_rawid, get_detector_system_for_channelname, get_filtered_keys_in_detectorsystem
from latools import core
from latools.browse import BrowseTask, BrowseAnydetTask
from latools.histogram import HistogramTask, Histogram2DTask, CategoricalHistogramTask, CategoricalHistogram2DTask, MultiarrayCategoricalHistogramTask
from latools.counter import CountTask

plt.rcParams["figure.figsize"] = (14, 4)
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["font.size"] = 14

proj_dir = "/mnt/atlas02/projects/legend/sipm_qc"
lmeta = LegendMetadata(os.path.join(proj_dir, "metadata/legend-metadata-schwarz"))
chmap = lmeta.channelmap("20250624T140504Z")

In [ ]:
old_dsp_dir = os.path.join(proj_dir, "manual_dsp/generated/p14r006dsp-dsp002")
#dsp_dir = os.path.join(proj_dir, "data/tier/dsp/lac/p14/r006")
old_dsp_files = glob.glob(old_dsp_dir+"/l200-*-tier_dsp.lh5")
old_dsp_files.sort()

new_dsp_dir = os.path.join(proj_dir, "manual_dsp/generated/p14r006dsp-dsp003")
#dsp_dir = os.path.join(proj_dir, "data/tier/dsp/lac/p14/r006")
new_dsp_files = glob.glob(new_dsp_dir+"/l200-*-tier_dsp.lh5")
new_dsp_files.sort()

In [ ]:
# These 49 rawids I see in the event tier of p14 r006 lac. Some channels existing in raw & dsp (e.g. S054) don't show up here...
rawids_used_in_evt = [1052803, 1052804, 1054401, 1054402, 1054403, 1054404, 1054405, 1056000, 1056001, 1056003, 1056004, 1056005, 1057601, 1057602, 1057603, 1057604, 1057605, 1059200, 1059201, 1059202, 1059204, 1059205, 1060801, 1060803, 1060804, 1060805, 1062400, 1062401, 1062402, 1062403, 1062404, 1062405, 1064000, 1064001, 1064002, 1064003, 1064004, 1065600, 1065601, 1065602, 1065603, 1065604, 1065605, 1067200, 1067201, 1067202, 1067203, 1067204, 1067205]
usable_sipm_keys = get_filtered_keys_in_detectorsystem(chmap, "spms", rawids_used_in_evt)
usable_sipm_keys.append("S058")
usable_sipm_keys.sort()

In [ ]:
# set up chain's input data
inputArraysDef = []

def generate_SiPM_hit_defs():
    ret = []
    for sipm in usable_sipm_keys:
        ret += [(f"{sipm}_energy", f"{sipm}/dsp/energy")] # per event and per pulse
    return ret
inputArraysDef += generate_SiPM_hit_defs()

In [ ]:
# set up processing functions
genArrayDef = []

def generate_SiPM_hit_gen_defs():
    ret = []
    for sipm in usable_sipm_keys:
        ret += [
            ([f"{sipm}_energy"], f"{sipm}_energy_sum", lambda x: ak.sum(x[0], axis=-1)) # per pulse
        ]
    return ret
genArrayDef += generate_SiPM_hit_gen_defs()

In [ ]:
old_dsptables = core.compile_arrays(inputArraysDef, genArrayDef, tier_filename_dict=[{"dsp": f} for f in old_dsp_files])
new_dsptables = core.compile_arrays(inputArraysDef, genArrayDef, tier_filename_dict=[{"dsp": f} for f in new_dsp_files])

In [ ]:
fig, ax = plt.subplots()
spm = "S057"
core.oneshot([old_dsptables[f"{spm}_energy"]], HistogramTask(-1, 20, 1000, lambda x: ak.flatten(x[0]), logy=True, ax=ax))
core.oneshot([new_dsptables[f"{spm}_energy"]], HistogramTask(-1, 20, 1000, lambda x: ak.flatten(x[0]), logy=True, ax=ax))

In [ ]:
fig, ax = plt.subplots()
spm = "S085"
core.oneshot([old_dsptables[f"{spm}_energy"]], HistogramTask(-1, 7, 1000, lambda x: ak.flatten(x[0]), logy=True, ax=ax))
core.oneshot([new_dsptables[f"{spm}_energy"]], HistogramTask(-1, 7, 1000, lambda x: ak.flatten(x[0]), logy=True, ax=ax))

In [ ]:
spm_iter = iter(usable_sipm_keys)
x = 6
y = 9
with plt.rc_context({"figure.figsize": (15, 18)}):
    fig, ax = plt.subplots(y, x)
for i in range(y):
    for j in range(x):
        ax[i][j].get_yaxis().set_visible(False)
        ax[i][j].get_xaxis().set_visible(False)
        try:
            spm = next(spm_iter)
        except StopIteration:
            continue
        core.oneshot([old_dsptables[f"{spm}_energy"]], HistogramTask(-0.1, 6, 1000, lambda x: ak.flatten(x[0]), logy=True, ax=ax[i][j]))
        core.oneshot([new_dsptables[f"{spm}_energy"]], HistogramTask(-0.1, 6, 1000, lambda x: ak.flatten(x[0]), logy=True, ax=ax[i][j]))
        ax[i][j].text(0.95, 0.99, spm,
            verticalalignment='top', horizontalalignment='right',
            transform=ax[i][j].transAxes)
            #color='green', fontsize=15)
fig.subplots_adjust(wspace=0, hspace=0)

In [ ]:
len(usable_sipm_keys)